In [7]:
import cv2

# Open the RTSP stream
cap = cv2.VideoCapture("rtsp://10.227.207.170:8554/webcam.sdp")

# Check if the stream opened successfully
if not cap.isOpened():
    print("Error: Could not open RTSP stream")
else:
    print("RTSP stream opened successfully")
    
    # Display the video stream
    while True:
        ret, frame = cap.read()
        
        if not ret:
            print("Error: Could not read frame")
            break
        
        # Display the frame
        cv2.imshow('RTSP Stream', frame)
        
        # Break the loop on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the capture and close windows
    cap.release()
    cv2.destroyAllWindows()

RTSP stream opened successfully


KeyboardInterrupt: 